<div align="center" style="background-color: #f8f9fa; padding: 20px; border: 1px solid #dee2e6; border-radius: 10px;">
    <h1 style="color: #2c3e50; border-bottom: none;"> Elektrik Tüketim Analizi </h1>
    <p style="color: #7f8c8d; font-style: italic;">İlçe Bazlı Tüketim ve Tahsilat Verileri Keşifsel Analizi</p>
</div>

###  Notebook Özeti
Bu notebook, **Hamamözü, Gümüşhacıköy ve Göynücek** ilçelerine ait elektrik verilerini kapsar.

* **Veri Seti:** `elektrik_veri_hashed.xlsx` dosyası üzerinden 5 farklı sayfa (Tahsilat ve Tahakkuk verileri) işlenmiştir.
* **İşlem Adımları:** * `fillna(0)` ile eksik verilerin temizlenmesi.
    * Aylık ve mevsimsel bazda `kwh` tüketim ortalamalarının hesaplanması.
    * IQR (Interquartile Range) yöntemiyle aykırı değerlerin tespiti.
    * Ödeme kanallarının (Nakit, Banka, Kart) ağırlıklarının belirlenmesi.
    * İlçe bazında kwh tüketimi istatistikleri ve tüketim miktarına göre sınıflama yapılması.
    * Müşterilerin Hesap sınıflarının adet ve dağılımlarının,tükettikleri ortalama kwh değerlerinin ve istatistiklerinin hesaplanması.

In [ ]:
import numpy as np
import pandas as pd

file_path = 'elektrik_veri_hashed.xlsx'
xls = pd.ExcelFile(file_path)


In [ ]:
print('Sayfalar: ',xls.sheet_names)

In [ ]:
df_tahsilat = pd.read_excel(xls,sheet_name='Tahsilat')
df_tahsilat_1 = pd.read_excel(xls,sheet_name='Tahsilat 1')
df_tahakkuk = pd.read_excel(xls,sheet_name='Tahakkuk')
df_tahakkuk_1 =pd.read_excel(xls,sheet_name='Tahakkuk 1')
df_tahakkuk_2= pd.read_excel(xls,sheet_name='Tahakkuk 2')

In [ ]:
print("Tahsilat Boyutu:", df_tahsilat.shape, "| Tahsilat 1 Boyutu:", df_tahsilat_1.shape,
 "| Tahakkuk Boyutu: ",df_tahakkuk.shape,"| Tahakkuk 1 Boyutu: ",df_tahakkuk_1.shape,
"| Tahakkuk 2 Boyutu: ",df_tahakkuk_2.shape)

In [ ]:
#%%Tahsilat 
df_tahsilat.head()
df_tahsilat.info()

In [ ]:
# Tüm NaN değerlerini 0 yap
df_tahsilat['Nakit Tahsilat'].isna().sum()
df_tahsilat['Nakit Tahsilat'] = df_tahsilat['Nakit Tahsilat'].fillna(0)

In [ ]:
df_tahsilat['Mahsuben Tahsilat'].isna().sum()
df_tahsilat['Mahsuben Tahsilat'] = df_tahsilat['Mahsuben Tahsilat'].fillna(0)

In [ ]:
df_tahsilat['Kredi Kartı Tahsilatı'].isna().sum()
df_tahsilat['Kredi Kartı Tahsilatı'] = df_tahsilat['Kredi Kartı Tahsilatı'].fillna(0)

In [ ]:
df_tahsilat['Banka Tahsilatı'].isna().sum()
df_tahsilat['Banka Tahsilatı'] = df_tahsilat['Banka Tahsilatı'].fillna(0)

In [ ]:
df_tahsilat.describe(include='all')
df_tahsilat['Toplam_Tahsilat_Tutari'] = df_tahsilat[['Nakit Tahsilat','Kredi Kartı Tahsilatı','Banka Tahsilatı','Mahsuben Tahsilat']].sum(axis=1)

In [ ]:
islem_sayilari = (df_tahsilat[['Banka Tahsilatı', 'Nakit Tahsilat', 'Mahsuben Tahsilat', 'Kredi Kartı Tahsilatı']] > 0).sum()
print(islem_sayilari)

In [ ]:
#%% Tahsilat 1
df_tahsilat_1.head()
df_tahsilat_1.info()
df_tahsilat_1['Mali yıl/dönem'].value_counts()

In [ ]:
aylar = {
    'OCK': 'Ocak',
    'ŞBT': 'Şubat',
    'MAR': 'Mart',
    'NİS': 'Nisan',
    'MAY': 'Mayıs',
    'HAZ': 'Haziran',
    'TEM': 'Temmuz',
    'AGU': 'Ağustos',
    'EYL': 'Eylül',
    'EKM': 'Ekim',
    'KSM': 'Kasım',
    'ARL': 'Aralık'
}


In [ ]:
df_tahsilat_1['Mali yıl/dönem'] = (
    df_tahsilat_1['Mali yıl/dönem']
    .astype(str)
    .str.strip()
    .replace(aylar, regex=True)
)

In [ ]:
#Tüm nan değerleri 0 yap
df_tahsilat_1.isna().sum()
df_tahsilat_1=df_tahsilat_1.fillna(0)

In [ ]:
# Sütun gruplarını tanımla
vaktinde_sutunlar = ['Son Ödeme Tarihinden Önceki Tahsilat', 'Son Ödeme Tarihindeki Tahsilat']
gecikmeli_sutunlar = [i for i in df_tahsilat_1.columns if 'Son Ödeme (' in i]

# Vaktinde ödenen kayıtlar (En az bir 'vaktinde' sütunu 0'dan büyük olan satırlar)
vaktinde_adet = df_tahsilat_1[df_tahsilat_1[vaktinde_sutunlar].gt(0).any(axis=1)].shape[0]

# Gecikmeli ödenen kayıtlar (En az bir 'gecikmeli' sütunu 0'dan büyük olan satırlar)
gecikmeli_adet = df_tahsilat_1[df_tahsilat_1[gecikmeli_sutunlar].gt(0).any(axis=1)].shape[0]

# Toplam işlem sayısı
toplam_islem = len(df_tahsilat_1)

#Yüzdeler
vaktinde_yuzde = (vaktinde_adet / toplam_islem) * 100
gecikmeli_yuzde = (gecikmeli_adet / toplam_islem) * 100

In [ ]:
print(f"Toplam Kayıt: {toplam_islem}")
print(f"Zamanında Ödenen Kayıt Oranı: %{vaktinde_yuzde:.2f}")
print(f"Gecikmeli Ödenen Kayıt Oranı: %{gecikmeli_yuzde:.2f}")

In [ ]:
pd.set_option('display.max_columns', None) #tüm sütunları göster
pd.set_option('display.max_rows', None) #tüm satırları göster

In [ ]:
df_tahsilat_1.describe(include='all')

In [ ]:
#%% Tahakkuk Hamamözü
df_tahakkuk.head()
df_tahakkuk.info()
df_tahakkuk['kwh'].describe()

In [ ]:
#tarih dönüşümü
df_tahakkuk['fatura_tarihi'] = pd.to_datetime(df_tahakkuk['fatura_tarihi'])
df_tahakkuk['kayit_tarihi']  = pd.to_datetime(df_tahakkuk['kayit_tarihi'])
df_tahakkuk['vade_tarihi']   = pd.to_datetime(df_tahakkuk['vade_tarihi'])
df_tahakkuk['mali_yil_donem'] = pd.to_datetime(df_tahakkuk['mali_yil_donem'])
df_tahakkuk['mali_yil_donem'] = (df_tahakkuk['mali_yil_donem'].dt.to_period('M')) #Ocak 2023 formatı
df_tahakkuk ['ay'] = df_tahakkuk['mali_yil_donem'].dt.month 

In [ ]:
#eksik değer kontrolü
eksikler = df_tahakkuk['kwh'].isna().sum()
print(eksikler)

In [ ]:
negatifler = df_tahakkuk[df_tahakkuk['kwh']<0]
negatif_sayisi = negatifler.shape[0] 
print(negatif_sayisi)                                       

In [ ]:
#aykırı değerler sınırı tespiti
q1 = df_tahakkuk['kwh'].quantile(0.25)
q3 = df_tahakkuk['kwh'].quantile(0.75)
iqr = q3-q1

In [ ]:
alt_sinir = q1-1.5*iqr
ust_sinir = q3 + 1.5*iqr

In [ ]:
uc_degerler = df_tahakkuk[(df_tahakkuk['kwh'] < alt_sinir) | (df_tahakkuk['kwh'] > ust_sinir)]
uc_deger_sayisi = len(uc_degerler)
print(uc_deger_sayisi)

In [ ]:
df_tahakkuk['sozlesme_hesap_no'].value_counts().sum() #müşteri sayısı

In [ ]:
df_tahakkuk.groupby('ilce')['sozlesme_hesap_no'].nunique().sort_values(ascending=False) #benzersiz müşteri

In [ ]:
ort_kw = df_tahakkuk['kwh'].mean()
medyan = df_tahakkuk['kwh'].median()

In [ ]:
print('------------Hamamözü Genel İstatistikler----------')
print(f"Eksik Kayıt Sayısı: {eksikler}")
#print(f"Negatif Kayıt Sayısı: {negatif_sayisi}")
print(f"Aşırı Uç (Outlier) Kayıt Sayısı: {uc_deger_sayisi}")
print("Benzersiz Müşteri Sayısı: ",df_tahakkuk.groupby('ilce')['sozlesme_hesap_no'].nunique().sort_values(ascending=False))
print(f"Normal Tüketim Üst Sınırı (Hesaplanan): {ust_sinir:.2f} kWh")
print(f'Ortalama Elektrik tüketimi: {ort_kw:.2f} kWh ')
print(f'Medyan: {medyan:.2f} kWh ')

In [ ]:
#%% Tahakkuk 1 Gümüşhacıköy
df_tahakkuk_1.head()
df_tahakkuk_1.info()
df_tahakkuk_1['kwh'].describe()

In [ ]:
df_tahakkuk_1['fatura_tarihi'] = pd.to_datetime(df_tahakkuk_1['fatura_tarihi'])
df_tahakkuk_1['kayit_tarihi']  = pd.to_datetime(df_tahakkuk_1['kayit_tarihi'])
df_tahakkuk_1['vade_tarihi']   = pd.to_datetime(df_tahakkuk_1['vade_tarihi'])
df_tahakkuk_1['mali_yil_donem'] = pd.to_datetime(df_tahakkuk_1['mali_yil_donem'])
df_tahakkuk_1['mali_yil_donem'] = (df_tahakkuk_1['mali_yil_donem'].dt.to_period('M')) 
df_tahakkuk_1 ['ay'] = df_tahakkuk_1['mali_yil_donem'].dt.month

In [ ]:
eksikler_1 = df_tahakkuk_1['kwh'].isna().sum()
print(eksikler_1)

In [ ]:
negatifler_1 = df_tahakkuk_1[df_tahakkuk_1['kwh']<0]
negatif_sayisi_1 =negatifler_1.shape[0]

In [ ]:
q1_1 = df_tahakkuk_1['kwh'].quantile(0.25)
q3_1 = df_tahakkuk_1['kwh'].quantile(0.75)
iqr_1 = q3_1 - q1_1

In [ ]:
alt_sinir_1 = q1_1-1.5*iqr_1
ust_sinir_1 = q3_1 + 1.5*iqr_1


In [ ]:
uc_degerler_1 = df_tahakkuk_1[(df_tahakkuk_1['kwh'] < alt_sinir_1) | (df_tahakkuk_1['kwh'] > ust_sinir_1)]
uc_deger_sayisi_1 = len(uc_degerler_1)
print(uc_deger_sayisi_1)

In [ ]:
df_tahakkuk_1['sozlesme_hesap_no'].value_counts().sum() #müşteri sayısı

In [ ]:
df_tahakkuk_1.groupby('ilce')['sozlesme_hesap_no'].nunique().sort_values(ascending=False) #benzersiz müşteri

In [ ]:
ort_kw_1 = df_tahakkuk_1['kwh'].mean()
medyan_1 = df_tahakkuk_1['kwh'].median()

In [ ]:
print('---------Gümüşhacıköy Genel İstatistikler--------')
print(f"Eksik Kayıt Sayısı: {eksikler_1}")
#print(f"Negatif Kayıt Sayısı: {negatif_sayisi_1}")
print("Benzersiz Müşteri Sayısı: ",df_tahakkuk_1.groupby('ilce')['sozlesme_hesap_no'] \
             .nunique() \
             .sort_values(ascending=False))
print(f"Aşırı Uç (Outlier) Kayıt Sayısı: {uc_deger_sayisi_1}")
print(f"Normal Tüketim Üst Sınırı (Hesaplanan): {ust_sinir_1:.2f} kWh")
print(f'Ortalama Elektrik tüketimi: {ort_kw_1:.2f} kWh ')
print(f'Medyan: {medyan_1:.2f} kWh ')

In [ ]:
#%% Tahakkuk 2 Göynücek
df_tahakkuk_2.head()
df_tahakkuk_2.info()
df_tahakkuk_2['kwh'].describe()

In [ ]:
df_tahakkuk_2['fatura_tarihi'] = pd.to_datetime(df_tahakkuk_2['fatura_tarihi'])
df_tahakkuk_2['kayit_tarihi']  = pd.to_datetime(df_tahakkuk_2['kayit_tarihi'])
df_tahakkuk_2['vade_tarihi']   = pd.to_datetime(df_tahakkuk_2['vade_tarihi'])
df_tahakkuk_2['mali_yil_donem'] = pd.to_datetime(df_tahakkuk_2['mali_yil_donem'])
df_tahakkuk_2['mali_yil_donem'] = (df_tahakkuk_2['mali_yil_donem'].dt.to_period('M')) 
df_tahakkuk_2['ay'] = df_tahakkuk_2['mali_yil_donem'].dt.month

In [ ]:
eksikler_2 = df_tahakkuk_2['kwh'].isna().sum()
print(eksikler_2)

In [ ]:
negatifler_2 = df_tahakkuk_2[df_tahakkuk_2['kwh']<0]
negatif_sayisi_2 =negatifler_2.shape[0]
print(negatif_sayisi_2)

In [ ]:
q1_2 = df_tahakkuk_2['kwh'].quantile(0.25)
q3_2 = df_tahakkuk_2['kwh'].quantile(0.75)
iqr_2 = q3_2 - q1_2

In [ ]:
alt_sinir_2 = q1_2 -1.5* iqr_2
ust_sinir_2 = q3_2 + 1.5*iqr_2

In [ ]:
uc_degerler_2 = df_tahakkuk_2[(df_tahakkuk_1['kwh'] < alt_sinir_2) | (df_tahakkuk_2['kwh'] > ust_sinir_2)]
uc_deger_sayisi_2 = len(uc_degerler_2)
print(uc_deger_sayisi_2)

In [ ]:
df_tahakkuk_2['sozlesme_hesap_no'].value_counts().sum() #müşteri sayısı

In [ ]:
df_tahakkuk_2.groupby('ilce')['sozlesme_hesap_no'].nunique().sort_values(ascending=False) #benzersiz müşteri

In [ ]:
ort_kw_2 = df_tahakkuk_2['kwh'].mean()
medyan_2 = df_tahakkuk_2['kwh'].median()

In [ ]:
print('---------Göynücek Genel İstatistikler--------')
print(f"Eksik Kayıt Sayısı: {eksikler_2}")
#print(f"Negatif Kayıt Sayısı: {negatif_sayisi_2}")
print("Benzersiz Müşteri Sayısı: ",df_tahakkuk_2.groupby('ilce')['sozlesme_hesap_no'] \
             .nunique() \
             .sort_values(ascending=False))
print(f"Aşırı Uç (Outlier) Kayıt Sayısı: {uc_deger_sayisi_2}")
print(f"Normal Tüketim Üst Sınırı (Hesaplanan): {ust_sinir_2:.2f} kWh")
print(f'Ortalama Elektrik tüketimi: {ort_kw_2:.2f} kWh ')
print(f'Medyan: {medyan_2:.2f} kWh ')

In [ ]:
#%%  birleştirme ve istatistikler
data = pd.concat(
    [df_tahakkuk, df_tahakkuk_1, df_tahakkuk_2],
    ignore_index=True)

In [ ]:
#tarih dönüşümü
data['fatura_tarihi'] = pd.to_datetime(data['fatura_tarihi'])
data['kayit_tarihi']  = pd.to_datetime(data['kayit_tarihi'])
data['vade_tarihi']   = pd.to_datetime(data['vade_tarihi'])

In [ ]:
# ilçe ve dönem bazlı istatistikler
data['ilce'].value_counts()
data.groupby('ilce')['kwh'].mean()
data.groupby('ilce')['kwh'].max()
data.groupby('mali_yil_donem')['kwh'].mean() #tüm ilçelerde dönem bazlı kwh ortalaması

In [ ]:
#en yaygın hesap sınıfları ve ort kwh
adet = data['Hesap Sınıfı'].value_counts()
yuzde = data['Hesap Sınıfı'].value_counts(normalize=True) * 100

In [ ]:
sonuc = pd.DataFrame({
    'Adet': adet,
    'Yüzde (%)': yuzde.round(2) 
})
print(sonuc)


In [ ]:
#hesap sınıfı özet istatistikler
hesap_sinifi_ozet =(
    data.groupby('Hesap Sınıfı')['kwh']
    .agg(
        Ort_kWh='mean',
        Medyan='median',
        Standart_Sapma='std'
    )
    .sort_values(by='Ort_kWh', ascending=False)
    .round(2))

print(hesap_sinifi_ozet)


In [ ]:
#ilçe mali yıl dönem bazında kwh 
ilce_mali_yil = (
data.groupby(['mali_yil_donem', 'ilce'], as_index=False)['kwh'].mean()
.rename(columns={'kwh': 'ortalama_kwh'}))

ilce_ay =pd.DataFrame(ilce_mali_yil)
print(ilce_mali_yil.round(2))


In [ ]:
#ilçe ay bazında kwh 
ilce_ay = (
data.groupby(['ay', 'ilce'], as_index=False)['kwh'].mean()
.rename(columns={'kwh': 'ortalama_kwh'}))

ilce_ay =pd.DataFrame(ilce_ay)
print(ilce_ay.round(2))


In [ ]:
#temmuz ayı ortalama kwh tüketimi
temmuz_ort = (data[(data['mali_yil_donem'].dt.month == 7) &
(data['mali_yil_donem'].dt.year.between(2023, 2025))]
    .groupby('ilce')['kwh']
    .mean()
    .sort_values(ascending=False))
print(temmuz_ort)

In [ ]:
#mevsime göre kwh tüketimi
data['Mevsim'] = data['ay'].map({
    12: 'Kış', 1: 'Kış', 2: 'Kış',
    3: 'İlkbahar', 4: 'İlkbahar', 5: 'İlkbahar',
    6: 'Yaz', 7: 'Yaz', 8: 'Yaz',
    9: 'Sonbahar', 10: 'Sonbahar', 11: 'Sonbahar'
})

mevsim = (data.groupby(['ilce','Mevsim'])['kwh'].agg(Ort_kWh='mean', Medyan='median', Standart_Sapma='std')
.sort_values(by=['ilce', 'Ort_kWh'], ascending=[True, False])
.round(2))

In [ ]:
mevsim = pd.DataFrame(mevsim)
print(mevsim)

In [ ]:
#kWh Tüketim Durumu
aylik_ort = data.groupby('ay')['kwh'].mean().reset_index(name='Ort_kWh').round(2)

max_kwh = aylik_ort['Ort_kWh'].max()
min_kwh = aylik_ort['Ort_kWh'].min()

In [ ]:
def durum_rapor(x):
    if x == max_kwh:
        return 'En Yüksek'
    elif x == min_kwh:
        return 'Düşük'
    elif x > aylik_ort['Ort_kWh'].mean():
        return 'Yüksek'
    else:
        return 'Normal'

In [ ]:
ay_map = {
    1:'Ocak', 2:'Şubat', 3:'Mart', 4:'Nisan',
    5:'Mayıs', 6:'Haziran', 7:'Temmuz', 8:'Ağustos',
    9:'Eylül', 10:'Ekim', 11:'Kasım', 12:'Aralık'
}

In [ ]:
aylik_ort['ay'] = aylik_ort['ay'].map(ay_map)
aylik_ort['Durum'] = aylik_ort['Ort_kWh'].apply(durum_rapor)
print(aylik_ort)
aylik_ort =pd.DataFrame(aylik_ort)


In [ ]:
data.to_csv('elektrik_verisi_temiz.csv') #temizlenen veriyi kaydetme